# 作業 : (Kaggle)鐵達尼生存預測
***
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[6], Out[6])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[10], Out[10], In[11], Out[11]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
df.dtypes

Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
dtype: object

In [3]:
df.nunique()

Pclass        3
Name        891
Sex           2
Age          88
SibSp         7
Parch         7
Ticket      681
Fare        248
Cabin       147
Embarked      3
dtype: int64

In [4]:
# Embarked 有空值
df['Embarked'].drop_duplicates()

0       S
1       C
5       Q
61    NaN
Name: Embarked, dtype: object

In [5]:
# Fare 沒有空值
df[df['Fare'].isna()]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [6]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
# 取'Embarked' 與 'Fare'
df['Embarked'] = df['Embarked'].fillna('None')

mean_df = df.groupby(['Embarked'])['Fare'].mean().reset_index().rename(columns={'Fare':'Fare_Mean'})
mode_df = df.groupby(['Embarked'])['Fare'].apply(lambda x:x.mode()[0]).reset_index().rename(columns={'Fare':'Fare_Mode'})
median_df = df.groupby(['Embarked'])['Fare'].median().reset_index().rename(columns={'Fare':'Fare_Median'})
max_df = df.groupby(['Embarked'])['Fare'].max().reset_index().rename(columns={'Fare':'Fare_Max'})
min_df = df.groupby(['Embarked'])['Fare'].min().reset_index().rename(columns={'Fare':'Fare_Min'})

temp = pd.merge(mean_df, mode_df, how='left', on=['Embarked'])
temp = pd.merge(temp, median_df, how='left', on=['Embarked'])
temp = pd.merge(temp, max_df, how='left', on=['Embarked'])
temp = pd.merge(temp, min_df, how='left', on=['Embarked'])

temp.head()

,Embarked,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max,Fare_Min
0,C,59.954144,7.2292,29.70,512.3292,4.0125
1,None,80.000000,80.0000,80.00,80.0000,80.0000
2,Q,13.276030,7.7500,7.75,90.0000,6.7500
3,S,27.079812,8.0500,13.00,263.0000,0.0000


In [7]:
df = pd.merge(df, temp, how='left', on=['Embarked'])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max,Fare_Min
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,27.079812,8.0500,13.0,263.0000,0.0000
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,59.954144,7.2292,29.7,512.3292,4.0125
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,27.079812,8.0500,13.0,263.0000,0.0000
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,27.079812,8.0500,13.0,263.0000,0.0000
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,27.079812,8.0500,13.0,263.0000,0.0000


In [8]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

10 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max', 'Fare_Min']



,Pclass,Age,SibSp,Parch,Fare,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max,Fare_Min
0,3,22.0,1,0,7.2500,27.079812,8.0500,13.0,263.0000,0.0000
1,1,38.0,1,0,71.2833,59.954144,7.2292,29.7,512.3292,4.0125
2,3,26.0,0,0,7.9250,27.079812,8.0500,13.0,263.0000,0.0000
3,1,35.0,1,0,53.1000,27.079812,8.0500,13.0,263.0000,0.0000
4,3,35.0,0,0,8.0500,27.079812,8.0500,13.0,263.0000,0.0000


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?
> 沒有改善 反而有些微下降，需再試試其他數值與類別欄位做 群聚編碼(Group by Encoding)

In [9]:
# 不讓下面程式碼 出現以下警告
# DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.

# 因現在df都是數值欄位
for c in df.columns:
    df[c] = df[c].astype(float)

In [10]:
# 沒有這五個新特徵的 dataframe 稱為 df_minus
df_minus = df.drop(['Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max', 'Fare_Min'] , axis=1)

# 原始特徵 + 邏輯斯迴歸
train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression(solver='liblinear')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7038635542329971

In [11]:
# 新特徵 + 邏輯斯迴歸
train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression(solver='liblinear')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7016289882888318